In [2]:
%%capture

# Cloning the repository and installing the required packages
!rm -rf liiondb
!git clone https://github.com/ndrewwang/liiondb.git
!pip uninstall -y sqlalchemy ipython-sql google-adk
!pip install sqlalchemy==1.4.51 pandas numpy matplotlib


# Import necessary libraries
import matplotlib.pyplot as plt
import numpy as np
import liiondb.functions.fn_db as fn_db
import pandas as pd
from sqlalchemy import create_engine

# Initialize the database connection using the liiondb function
dfndb, db_connection = fn_db.liiondb()

# Ensure that dfndb is a proper SQLAlchemy engine
engine = create_engine(dfndb.url)  # Create an engine from the connection URL

In [3]:
## Fetching the parameters from liiondb database for Positive electrode LFP.
QUERY = ''' SELECT DISTINCT data.data_id, parameter.name AS parameter,
            material.name AS material, data.raw_data, parameter.units_output,
            paper.paper_tag, data.raw_data_class, data.function,data.input_range, paper.doi
    FROM data
    JOIN paper ON paper.paper_id = data.paper_id
    JOIN material ON material.material_id = data.material_id
    JOIN parameter ON parameter.parameter_id = data.parameter_id
    WHERE material.name = 'LFP'
        '''
with engine.connect() as connection:
    result = connection.execute(QUERY)
    df = pd.DataFrame(result.fetchall(), columns=result.keys())

/tmp/ipython-input-2661675416.py:12: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  result = connection.execute(QUERY)


In [4]:
df.head(5)

,data_id,parameter,material,raw_data,units_output,paper_tag,raw_data_class,function,input_range,doi
0,313,half cell ocv,LFP,see function,V,Kashkooli2016,function,"[b'\n', b'd', b'e', b'f', b' ', b'f', b'u', b'...","[0, 0.95]",10.1016/j.jpowsour.2015.12.134
1,314,diffusion coefficient,LFP,see function,m^2*s^-1,Kashkooli2016,function,"[b'\n', b'd', b'e', b'f', b' ', b'f', b'u', b'...","[0, 0.95]",10.1016/j.jpowsour.2015.12.134
2,317,porosity,LFP,0.4,none,Kashkooli2016,value,None,"[NaN, NaN]",10.1016/j.jpowsour.2015.12.134
3,318,tortuosity,LFP,2.25,none,Kashkooli2016,value,None,"[NaN, NaN]",10.1016/j.jpowsour.2015.12.134
4,319,electronic conductivity,LFP,0.03,S*m^-1,Kashkooli2016,value,None,"[NaN, NaN]",10.1016/j.jpowsour.2015.12.134


In [5]:
df.shape

(80, 10)

In [6]:
from liiondb.streamlit_gui.elements import parameter_from_db as param
import os
# function to fetch the parameters in function format from "df" dataframe
def write_file(function_binary, write_file_path):
    os.makedirs(os.path.dirname(write_file_path), exist_ok=True)
    with open(write_file_path, 'wb') as f:
        f.write(function_binary)

In [7]:
# function to fetch the parameters in array format from "df" dataframe
def read_data(df):
    import numpy as np
    import os
    raw_data = df
    csv_array = raw_data
    csv_array = csv_array.replace("{", "[")
    csv_array = csv_array.replace("}", "]")
    csv_list = eval(csv_array)
    raw_data = csv_list
    raw_data = np.array(raw_data)

    return raw_data

In [8]:
for i in range(df.shape[0]):
  if df['raw_data_class'][i]=='function':
    write_file(df['function'][i],"content/streamlit_gui/elements/"+df['parameter'][i]+df['paper_tag'][i]+".py") #use to store and download the functions

In [9]:
LFP_HOCVs=[] # list to store the HOCVs of LFP in array format
LFP_Paper_tags=[] # list of corresponding paper tags

for i in range(df.shape[0]):
  if (df['raw_data_class'][i]=='array') & (df['parameter'][i]=='half cell ocv'):
    csv_data=read_data(df['raw_data'][i])
    LFP_HOCVs.append(csv_data)
    LFP_Paper_tags.append(df['paper_tag'][i])
    print(df['paper_tag'][i])

Prada2012
Prada2012
Srinivasan2004a


In [10]:
## Fetching the parameters from liiondb database for Negative electrode Graphite.
QUERY = '''
    SELECT DISTINCT data.data_id, parameter.symbol, parameter.name AS parameter,
            material.name AS material, data.raw_data, parameter.units_output,
            paper.paper_tag, data.raw_data_class, data.function,data.input_range, paper.doi
    FROM data
    JOIN paper ON paper.paper_id = data.paper_id
    JOIN material ON material.material_id = data.material_id
    JOIN parameter ON parameter.parameter_id = data.parameter_id
    WHERE parameter.name = 'half cell ocv'
    AND material.name = 'Graphite'
'''

# Use the engine to execute the query and read the results into a DataFrame
with engine.connect() as connection:
    result = connection.execute(QUERY)
    df = pd.DataFrame(result.fetchall(), columns=result.keys())

In [11]:
Graphite_HOCVs=[] # list to store the HOCVs of Graphite in array format
Graphite_Paper_tags=[] # list of corresponding paper tags

for i in range(df.shape[0]):
  if (df['raw_data_class'][i]=='array') & (df['parameter'][i]=='half cell ocv'):
    csv_data=read_data(df['raw_data'][i])
    Graphite_HOCVs.append(csv_data)
    Graphite_Paper_tags.append(df['paper_tag'][i])
    print(df['paper_tag'][i])

Ecker2015
Schmalstieg2018
Prada2012
Prada2012
Liebig2019
Birkl2015
Birkl2015
Birkl2015
Li2012
Hust2019
Dufour2018
Dufour2018
Kumaresan2008
Chaouachi2021


In [12]:
exclude_tags = {
    'Birkl2015',
    'Li2012',
    'Kumaresan2008',
    'Schmalstieg2018',
    'Liebig2019',
    'Hust2019',
    'Dufour2018'
}
df_graphite = pd.DataFrame({'Graphite_HOCVs': Graphite_HOCVs, 'Graphite_Paper_tags': Graphite_Paper_tags}) # Grpahite electrode HOCVs dataframe
df_graphite = df_graphite[~df_graphite['Graphite_Paper_tags'].isin(exclude_tags)] # Filtering the certain paper tags
Graphite_HOCVs_filtered=df_graphite['Graphite_HOCVs'].to_list() # Filtered Grpahite electrode HOCVs list

In [13]:
df_graphite

,Graphite_HOCVs,Graphite_Paper_tags
0,"[[0.0015151515151514696, 1.4325153374233128], ...",Ecker2015
2,"[[0.023507969717231403, 0.4813169984686064], [...",Prada2012
3,"[[0.023689149, 0.57136294], [0.050050687, 0.37...",Prada2012
13,"[[0.00972762645914399, 0.8609112709832133], [0...",Chaouachi2021
